## Applying Energy Path algtm to road photos

In [1]:
import cv2
import numpy as np
import matplotlib as plt

In [2]:
RED = (0, 0, 255)

In [3]:
img = cv2.imread("C:\Programs/facultate/licenta/my-autonomous-car/resources/dottedStraight.png")


In [4]:
print(img)

[[[184 183 165]
  [226 225 207]
  [224 223 205]
  ...
  [127 131 126]
  [118 122 117]
  [109 113 107]]

 [[183 184 165]
  [241 242 224]
  [251 251 233]
  ...
  [161 165 160]
  [152 156 151]
  [144 148 142]]

 [[182 185 167]
  [238 241 222]
  [255 255 247]
  ...
  [ 99 103  98]
  [ 88  92  86]
  [ 90  94  89]]

 ...

 [[ 60  62  62]
  [ 60  62  62]
  [ 60  62  62]
  ...
  [ 43  46  45]
  [ 43  46  45]
  [ 43  46  45]]

 [[ 60  62  62]
  [ 60  62  62]
  [ 60  62  62]
  ...
  [ 43  46  45]
  [ 43  46  45]
  [ 43  46  45]]

 [[ 60  62  62]
  [ 60  62  62]
  [ 60  62  62]
  ...
  [ 43  46  45]
  [ 43  46  45]
  [ 43  46  45]]]


In [5]:

cv2.imshow("original",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("gray",gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
blurred = cv2.GaussianBlur(gray,(3,3),3)

cv2.imshow("blurred",blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Bilateral blur


In [21]:
bilateralBlurred = cv2.bilateralFilter(gray, 11, 21, 7)

cv2.imshow("bilateral Blurred",bilateralBlurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
blurred = cv2.GaussianBlur(gray,(3,3),3)
bilateralBlurred = cv2.bilateralFilter(gray, 11, 21, 7)

cv2.imshow("bilateral Blurred",bilateralBlurred)


cv2.imshow("blurred",blurred)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Classic binary thresholding

In [8]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Adaptive binary thresholding

In [9]:
threshAdap = cv2.adaptiveThreshold(blurred, 255,
	cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 5)

cv2.imshow("adaptive thresh",threshAdap)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Morphological ops

In [10]:
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)
cv2.imshow("opened img",opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
_, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
# kernel = np.ones((3,3),np.uint8)
kernel = cv2.getGaussianKernel(5,5)
closing = cv2.morphologyEx(threshClassis, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

# cv2.imwrite("classicTresh.jpg",threshClassis)
cv2.imshow("classic thresh",threshClassis)
cv2.imshow("opened img",opening)
cv2.imshow("closed img",closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Gradient making

In [13]:
def compute_energy(img):
    """
    calculeaza energia la fiecare pixel pe baza gradientului
    :param img: imaginea initiala
    :return:E - energia
    """
    # urmati urmatorii pasi:
    # 1. transformati imagine in grayscale
    # 2. folositi filtru sobel pentru a calcula gradientul in directia X si Y
    # 3. calculati magnitudinea pentru fiecare pixel al imaginii
    E = np.zeros((img.shape[0],img.shape[1]))
    # img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    grad_x = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=1,dy=0)
    grad_y = cv2.Sobel(img,ddepth=cv2.CV_16S,dx=0,dy=1)

    abs_x = np.abs(grad_x)
    abs_y = np.abs(grad_y)

    E = abs_x + abs_y

    return E

In [14]:
gradient = compute_energy(opening)
cv2.imshow("gradient img",gradient)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
def select_dynamic_programming_path_going_up(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    print(np.max(M[line,:]))
    path = [0 for i in range(line+1)]

    path[line] = (line,col)
    
    for line in range(M.shape[0]-2,-1,-1):
        if col == 0:
            if M[line,0] < M[line,1]:
                new_col = 1
            else:
                new_col = 0
        elif col == E.shape[1] - 1:
            if M[line,col] < M[line,col-1]:
                new_col = col - 1
            else:
                new_col = col
        else:
            neigh = np.array([M[line,col-1],M[line,col],M[line,col+1]])
            new_col = col+np.argmax(neigh) - 1

        path[line]= (line,new_col)

        col = new_col

    return path

In [16]:
def select_dynamic_programming_path_going_sideways(E):
    M = np.zeros(E.shape)
    M[0,:] = E[0,:]
    for i in range(1,E.shape[0]):
        for j  in range (E.shape[1]):
            if j == 0:
                M[i][j] = E[i,j]+max(M[i-1,0],M[i-1,1])
            elif j == E.shape[1] - 1:
                M[i,j] = E[i,j]+max(M[i-1,j],M[i-1,j-1])
            else:
                M[i,j] = E[i,j]+max(M[i-1,j+1],M[i-1,j],M[i-1,j-1],M[i][j+1],M[i][j-1])
    line = M.shape[0] - 1
    col = np.argmax(M[line,:])
    print(np.max(M[line,:]))
    path = [0 for _ in range(60)]
    # path = []
    index = 0

    path[index] = (line,col)

    lineIndex = M.shape[0]-1
    nextLine = False
    while lineIndex > 0 and col >= 0 and col < M.shape[1] and index != 50:
        if col == 0:
            # if M[lineIndex,0] < M[lineIndex,1]:
            #     new_col = 1
            #     nextLine = False
            # else:
            #     new_col = 0
            #     nextLine = False
            if M[lineIndex-1,0] < M[lineIndex-1,1]:
                new_col = 1
                nextLine = True
            else:
                new_col = 0
                nextLine = True
        elif col == E.shape[1] - 1:
            if M[lineIndex-1,col] < M[lineIndex-1,col-1]:
                new_col = col - 1
                nextLine = True
            else:
                new_col = col
                nextLine = True
        else:
            same_line = np.array([M[lineIndex,col-1],M[lineIndex,col+1]])
            upper_line = np.array([M[lineIndex-1,col-1],M[lineIndex-1,col],M[lineIndex-1,col+1]])
            if np.max(same_line) > np.max(upper_line):
                new_col = col+np.argmax(same_line) - 1
                nextLine = False
            else:
                new_col = col+np.argmax(upper_line) - 1
                nextLine = True

        if nextLine:
            lineIndex -= 1

        index += 1  
        path[index]= (lineIndex,new_col)

        col = new_col
    
    return path

In [17]:
def create_path(img, pathL,pathR, color):
    
    new_image = img.copy()

    for row, col in pathL:
        new_image[row, col] = color

    for row, col in pathR:
        new_image[row, col] = color

    # E = compute_energy(img)
    # new_image_E = img.copy()
    # new_image_E[:,:,0] = E.copy()
    # new_image_E[:,:,1] = E.copy()
    # new_image_E[:,:,2] = E.copy()

    # for row, col in path:
    #     new_image_E[row, col] = color
    # cv2.imshow(name, np.uint8(new_image))
    return new_image
    # cv2.imshow('path E', np.uint8(new_image_E))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [18]:
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray,(3,3),3)

    _, threshClassis = cv2.threshold(blurred, 185, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(threshClassis, cv2.MORPH_OPEN, kernel)

    return opening


In [19]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
cv2.imshow('PreR', preR)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Pathfinding going always up

In [20]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
gradL = compute_energy(preL)
gradR = compute_energy(preR)
pathL = select_dynamic_programming_path_going_up(gradL)
print(pathL)
pathR = select_dynamic_programming_path_going_up(gradR)
pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])

# cv2.imshow('preprocess img', pre)
cv2.imshow('gradientL img', gradL)
cv2.imshow('gradientR img', gradR)
new_img = create_path(img,pathL,pathR,RED)
cv2.imshow("New Image",new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

266220.0
[(0, np.int64(83)), (1, np.int64(84)), (2, np.int64(85)), (3, np.int64(86)), (4, np.int64(87)), (5, np.int64(88)), (6, np.int64(89)), (7, np.int64(90)), (8, np.int64(91)), (9, np.int64(92)), (10, np.int64(93)), (11, np.int64(94)), (12, np.int64(95)), (13, np.int64(96)), (14, np.int64(97)), (15, np.int64(98)), (16, np.int64(98)), (17, np.int64(98)), (18, np.int64(98)), (19, np.int64(98)), (20, np.int64(97)), (21, np.int64(96)), (22, np.int64(95)), (23, np.int64(94)), (24, np.int64(93)), (25, np.int64(92)), (26, np.int64(91)), (27, np.int64(90)), (28, np.int64(89)), (29, np.int64(88)), (30, np.int64(87)), (31, np.int64(86)), (32, np.int64(85)), (33, np.int64(84)), (34, np.int64(83)), (35, np.int64(82)), (36, np.int64(81)), (37, np.int64(80)), (38, np.int64(79)), (39, np.int64(78)), (40, np.int64(77)), (41, np.int64(78)), (42, np.int64(79)), (43, np.int64(80)), (44, np.int64(81)), (45, np.int64(82)), (46, np.int64(83)), (47, np.int64(84)), (48, np.int64(85)), (49, np.int64(86)), 

### Pathfinding going sideways as well

In [36]:
pre = preprocess(img)
preL = pre[:,:int(pre.shape[1]/2)]
preR = pre[:,int(pre.shape[1]/2):]
gradL = compute_energy(preL)
gradR = compute_energy(preR)
pathL = select_dynamic_programming_path_going_sideways(gradL)
print(pathL)
pathR = select_dynamic_programming_path_going_sideways(gradR)
pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
# # print(newPathR)
# # cv2.imshow('preprocess img', pre)
# cv2.imshow('gradientL img', gradL)
# cv2.imshow('gradientR img', gradR)
new_img = create_path(img,pathL,pathR,RED)
# cv2.imshow("New Image",new_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

266220.0
[(239, np.int64(0)), (238, 0), (237, 0), (236, 0), (235, 0), (234, 0), (233, 0), (232, 0), (231, 0), (230, 0), (229, 0), (228, 0), (227, 0), (226, 0), (225, 0), (224, 0), (223, 0), (222, 0), (221, 0), (220, 0), (219, 0), (218, 0), (217, 0), (216, 0), (215, 0), (214, 0), (213, 0), (212, 0), (211, 0), (210, 0), (209, 0), (208, 0), (207, 0), (206, 0), (205, 0), (204, 0), (203, 1), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), (202, np.int64(2)), 0, 0, 0, 0, 0, 0, 0, 0, 0]
555900.0


TypeError: cannot unpack non-iterable int object

In [ ]:
cap = cv2.VideoCapture('C:/Programs/facultate/licenta/my-autonomous-car/resources/1.mp4')
 
while cap.isOpened():
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    pre = preprocess(frame)
    preL = pre[:,:int(pre.shape[1]/2)]
    preR = pre[:,int(pre.shape[1]/2):]
    gradL = compute_energy(preL)
    gradR = compute_energy(preR)
    pathL = select_dynamic_programming_path_going_up(gradL)
    pathR = select_dynamic_programming_path_going_up(gradR)
    pathR =([(x,np.uint64(y + int(pre.shape[1]/2))) for x,y in pathR])
    new_img = create_path(frame,pathL,pathR,RED)
    cv2.imshow("pre",pre)
    cv2.imshow('newImg', new_img)
    
    if cv2.waitKey(1) == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()